# Trainer Test Run

## Set up

TFX requires apache-airflow and docker SDK.


In [1]:
!pip install 'apache-airflow[gcp]' docker tfx

You should consider upgrading via the 'pip install --upgrade pip' command.


In this notebook, we use TFX version 0.13.0

In [2]:
import tfx
tfx.version.__version__

'0.13.0'

TFX requires TensorFlow >= 1.13.1

In [3]:
import tensorflow as tf
tf.__version__

'1.13.1'

TFX supports Python 3.5 from version 0.13.0

In [4]:
import sys
sys.version

'3.5.2 (default, Nov 12 2018, 13:43:14) \n[GCC 5.4.0 20160609]'

## Download sample data

In [5]:
%%bash
# This enables you to run this notebook twice.
# There should not be train/eval files at ~/taxi/data, since TFX can handle only single file with version 0.13.0
if [ -e ~/taxi/data ]; then
    rm -rf ~/taxi/data
fi

# download taxi data
mkdir -p ~/taxi/data/simple
mkdir -p ~/taxi/serving_model/taxi_simple
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv -O ~/taxi/data/simple/data.csv

# download 
wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/taxi_utils.py -O ~/taxi/taxi_utils.py

--2019-06-25 09:05:01--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.108.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922668 (1.8M) [text/plain]
Saving to: ‘/root/taxi/data/simple/data.csv’

     0K .......... .......... .......... .......... ..........  2% 3.59M 0s
    50K .......... .......... .......... .......... ..........  5% 6.44M 0s
   100K .......... .......... .......... .......... ..........  7% 10.1M 0s
   150K .......... .......... .......... .......... .......... 10% 8.41M 0s
   200K .......... .......... .......... .......... .......... 13% 19.2M 0s
   250K .......... .......... .......... .......... .......... 15% 7.20M 0s
   300K .......... .......... .......... .......... .......... 18% 8.96M 0s
   350K ........

## Import

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import logging
import os
from google.protobuf import json_format

from tfx.components.base.base_component import ComponentOutputs
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.example_validator.component import ExampleValidator
from tfx.components.model_validator.component import ModelValidator
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.orchestration.airflow.airflow_runner import AirflowDAGRunner
from tfx.orchestration.pipeline import Pipeline
from tfx.orchestration.tfx_runner import TfxRunner
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import csv_input
from tfx.utils.channel import Channel
from tfx.utils import types

/usr/local/lib/python3.5/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


## configs

In [7]:
# This example assumes that the taxi data is stored in ~/taxi/data and the
# taxi utility function is in ~/taxi.  Feel free to customize this as needed.
_taxi_root = os.path.join(os.environ['HOME'], 'taxi')
_data_root = os.path.join(_taxi_root, 'data/simple')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_taxi_module_file = os.path.join(_taxi_root, 'taxi_utils.py')

# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_taxi_root, 'serving_model/taxi_simple')

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')
_metadata_db_root = os.path.join(_tfx_root, 'metadata')
_log_root = os.path.join(_tfx_root, 'logs')

# Airflow-specific configs; these will be passed directly to airflow
_airflow_config = {
    'schedule_interval': None,
    'start_date': datetime.datetime(2019, 1, 1),
}

# Logging overrides
logger_overrides = {'log_root': _log_root, 'log_level': logging.INFO}

## Create ExampleGen

In [8]:
"""Implements the chicago taxi pipeline with TFX."""
examples = csv_input(_data_root)

# Brings data into the pipeline or otherwise joins/converts training data.
train_config = example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=2)
eval_config = example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        train_config,
        eval_config
    ]))

# Create outputs
train_examples = types.TfxType(type_name='ExamplesPath', split='train')
train_examples.uri = os.path.join(_data_root, 'csv_example_gen/train/')

eval_examples = types.TfxType(type_name='ExamplesPath', split='eval')
eval_examples.uri = os.path.join(_data_root, 'csv_example_gen/eval/')

example_outputs = ComponentOutputs({
    'examples': Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[train_examples, eval_examples]
    ),
    'training_examples': Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[train_examples]
    ),
    'eval_examples': Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[eval_examples]
    ),    
})

example_gen = CsvExampleGen(
    input_base=examples, # A Channel of 'ExternalPath' type, it contains path of data source.
    output_config=output_config,  # An example_gen_pb2.Output instance, it contains train-eval split ratio.
    outputs=example_outputs # dict from name to output channel, it will be stored example_gen.outputs
)

## Create StatisticsGen

In [9]:
# Create outputs
train_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='train')
train_statistics.uri = os.path.join(_data_root, 'statistics_gen/train/')

eval_statistics = types.TfxType(type_name='ExampleStatisticsPath', split='eval')
eval_statistics.uri = os.path.join(_data_root, 'statistics_gen/eval/')

statistics_outputs = ComponentOutputs({
    'output': Channel(
        type_name='ExampleStatisticsPath',
        static_artifact_collection=[train_statistics, eval_statistics]
    )
})

statistics_gen = StatisticsGen(
    input_data=example_gen.outputs.examples, # A Channel of 'ExamplesPath' type, it is equal to example_outputs
    name='Statistics Generator', # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    outputs=statistics_outputs # dict from name to output channel, it will be stored statistics_gen.outputs
)

## Create SchemaGen

In [10]:
# Create outputs
train_schema_path = types.TfxType(type_name='SchemaPath', split='train')
train_schema_path.uri = os.path.join(_data_root, 'schema_gen/')

# NOTE: SchemaGen.executor can handle JUST ONE SchemaPath.
# Two or more SchemaPaths will cause ValueError
# such as "ValueError: expected list length of one but got 2".
schema_outputs = ComponentOutputs({
    'output':Channel(
        type_name='SchemaPath',
        static_artifact_collection=[train_schema_path] 
    )
})

infer_schema = SchemaGen(
    stats=statistics_gen.outputs.output, # A Channel of 'ExampleStatisticsPath' type, it is equal to statistics_outputs
    name='Schema Generator',  # Optional, name should be unique if you are going to use multiple StatisticsGen in same pipeline.
    outputs=schema_outputs # dict from name to output channel, it will be stored schema_gen.outputs
)

## Create Transform

In [11]:
train_examples = types.TfxType(type_name='ExamplesPath', split='train')
train_examples.uri = os.path.join(_data_root,
                                  'transform/transformed_examples/train/')
eval_examples = types.TfxType(type_name='ExamplesPath', split='eval')
eval_examples.uri = os.path.join(_data_root,
                                 'transform/transformed_examples/eval/')
transform_output = types.TfxType(type_name='TransformPath')
transform_output.uri = os.path.join(_data_root,
                                    'transform/transform_output/')

transform_outputs = ComponentOutputs({
    # Output of 'tf.Transform', which includes an exported 
    # Tensorflow graph suitable for both training and serving
    'transform_output':Channel(
        type_name='TransformPath',
        static_artifact_collection=[transform_output]
    ),
    # transformed_examples: Materialized transformed examples, which includes 
    # both 'train' and 'eval' splits.
    'transformed_examples':Channel(
        type_name='ExamplesPath',
        static_artifact_collection=[train_examples, eval_examples]
    )
})

transform = Transform(
    input_data=example_gen.outputs.examples,
    schema=infer_schema.outputs.output,
    module_file=_taxi_module_file,
    outputs=transform_outputs
)

## Create Trainer

In [12]:
model_exports = types.TfxType(type_name='ModelExportPath')
model_exports.uri = os.path.join(_data_root, 'trainer/current/')

trainer_outputs = ComponentOutputs({
    'output':Channel(
        type_name='ModelExportPath',
        static_artifact_collection=[model_exports]
    )
})

trainer = Trainer(
    module_file=_taxi_module_file,
    transformed_examples=transform.outputs.transformed_examples,
    schema=infer_schema.outputs.output,
    transform_output=transform.outputs.transform_output,
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000),
    outputs=trainer_outputs
)


## Create Model Analyzer

In [13]:
eval_output = types.TfxType('ModelEvalPath')
eval_output.uri = os.path.join(_data_root, 'eval_output/')

model_analyzer_outputs = ComponentOutputs({
    'output':
    Channel(
        type_name='ModelEvalPath',
        static_artifact_collection=[eval_output]),
})

feature_slicing_spec = evaluator_pb2.FeatureSlicingSpec(specs=[
    evaluator_pb2.SingleSlicingSpec(
        column_for_slicing=['trip_start_hour'])
])

model_analyzer = Evaluator(
    examples=example_gen.outputs.examples,
    model_exports=trainer.outputs.output,
    feature_slicing_spec=feature_slicing_spec,
    outputs=model_analyzer_outputs
)

In [14]:
def _Do(self, input_dict, output_dict, exec_properties):
    import apache_beam as beam
    import tensorflow as tf
    import tensorflow_model_analysis as tfma
    from typing import Any, Dict, List, Text
    from tfx.components.base import base_executor
    from tfx.proto import evaluator_pb2
    from tfx.utils import io_utils
    from tfx.utils import path_utils
    from tfx.utils import types
    from google.protobuf import json_format

    """Runs a batch job to evaluate the eval_model against the given input.
    Args:
      input_dict: Input dict from input key to a list of Artifacts.
        - model_exports: exported model.
        - examples: examples for eval the model.
      output_dict: Output dict from output key to a list of Artifacts.
        - output: model evaluation results.
      exec_properties: A dict of execution properties.
        - feature_slicing_spec: JSON string of evaluator_pb2.FeatureSlicingSpec
          instance, providing the way to slice the data.
    Returns:
      None
    """
    if 'model_exports' not in input_dict:
      raise ValueError('\'model_exports\' is missing in input dict.')
    if 'examples' not in input_dict:
      raise ValueError('\'examples\' is missing in input dict.')
    if 'output' not in output_dict:
      raise ValueError('\'output\' is missing in output dict.')

    self._log_startup(input_dict, output_dict, exec_properties)

    # Extract input artifacts
    model_exports_uri = types.get_single_uri(input_dict['model_exports'])

    feature_slicing_spec = evaluator_pb2.FeatureSlicingSpec()
    json_format.Parse(exec_properties['feature_slicing_spec'],
                      feature_slicing_spec)
    slice_spec = self._get_slice_spec_from_feature_slicing_spec(
        feature_slicing_spec)

    output_uri = types.get_single_uri(output_dict['output'])

    eval_model_path = path_utils.eval_model_path(model_exports_uri)

    tf.logging.info('Using {} for model eval.'.format(eval_model_path))
    eval_shared_model = tfma.default_eval_shared_model(
        add_metrics_callbacks=[
                        # calibration_plot_and_prediction_histogram computes calibration plot and prediction
                        # distribution at different thresholds.
                        tfma.post_export_metrics.calibration_plot_and_prediction_histogram(),
                        # auc_plots enables precision-recall curve and ROC visualization at different thresholds.
                        tfma.post_export_metrics.auc_plots()
                    ],
        eval_saved_model_path=eval_model_path)

    tf.logging.info('Evaluating model.')
    with beam.Pipeline(argv=self._get_beam_pipeline_args()) as pipeline:
      # pylint: disable=expression-not-assigned
      (pipeline
       | 'ReadData' >> beam.io.ReadFromTFRecord(
           file_pattern=io_utils.all_files_pattern(
               types.get_split_uri(input_dict['examples'], 'eval')))
       |
       'ExtractEvaluateAndWriteResults' >> tfma.ExtractEvaluateAndWriteResults(
           eval_shared_model=eval_shared_model,
           slice_spec=slice_spec,
           output_path=output_uri))
    tf.logging.info(
        'Evaluation complete. Results written to {}.'.format(output_uri))

model_analyzer.executor.Do = _Do

## Create Model Validator

## Create Pipeline

In [16]:
pipeline = Pipeline(
    pipeline_name="TFX Pipeline",
    pipeline_root=_pipeline_root,
    components=[example_gen, statistics_gen, infer_schema, transform, trainer, model_analyzer]
#     components=[model_analyzer]
)

## Execute

In [17]:
class DirectRunner(TfxRunner):
    """Tfx runner on local"""
    
    def __init__(self, config=None):
        self._config = config or {}
    
    def run(self, pipeline):
        for component in pipeline.components:
            self._execute_component(component)
            
        return pipeline
            
    def _execute_component(self, component):
        input_dict = {key:value.get() for key, value in component.input_dict.items()}
        output_dict = {key: value.get() for key, value in component.outputs.get_all().items()}
        exec_properties = component.exec_properties
        executor = component.executor()
        executor.Do(input_dict, output_dict, exec_properties)

In [18]:
pipeline = DirectRunner().run(pipeline)

INFO:tensorflow:Starting Executor execution.
[2019-06-25 09:05:04,737] {base_executor.py:72} INFO - Starting Executor execution.
INFO:tensorflow:Inputs for Executor is: {"input-base": [{"artifact": {"uri": "/root/taxi/data/simple", "properties": {"split": {"stringValue": ""}, "type_name": {"stringValue": "ExternalPath"}}}, "artifact_type": {"name": "ExternalPath", "properties": {"name": "STRING", "state": "STRING", "span": "INT", "split": "STRING", "type_name": "STRING"}}}]}
[2019-06-25 09:05:04,743] {base_executor.py:74} INFO - Inputs for Executor is: {"input-base": [{"artifact": {"uri": "/root/taxi/data/simple", "properties": {"split": {"stringValue": ""}, "type_name": {"stringValue": "ExternalPath"}}}, "artifact_type": {"name": "ExternalPath", "properties": {"name": "STRING", "state": "STRING", "span": "INT", "split": "STRING", "type_name": "STRING"}}}]}
INFO:tensorflow:Outputs for Executor is: {"training_examples": [{"artifact": {"uri": "/root/taxi/data/simple/csv_example_gen/train

[2019-06-25 09:05:12,103] {fn_api_runner.py:437} INFO - Running ((ShuffleSpliteval/ReshufflePerKey/GroupByKey/Read)+((ref_AppliedPTransform_ShuffleSpliteval/ReshufflePerKey/FlatMap(restore_timestamps)_60)+(ref_AppliedPTransform_ShuffleSpliteval/RemoveRandomKeys_61)))+(((ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/WriteBundles_68)+(ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/Pair_69))+((ref_AppliedPTransform_OutputSpliteval/Write/WriteImpl/WindowInto(WindowIntoFn)_70)+(OutputSpliteval/Write/WriteImpl/GroupByKey/Write)))
[2019-06-25 09:05:12,192] {tfrecordio.py:57} WARNING - Couldn't find python-snappy so the implementation of _TFRecordUtil._masked_crc32c is not as fast as it could be.
[2019-06-25 09:05:12,383] {fn_api_runner.py:437} INFO - Running ((ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/DoOnce/Read_40)+((ref_AppliedPTransform_OutputSplittrain/Write/WriteImpl/InitializeWrite_41)+(ref_PCollection_PCollection_25/Write)))+(ref_PCollection_PCollection_24/

[2019-06-25 09:05:17,739] {fn_api_runner_transforms.py:490} INFO - ==================== <function read_to_impulse at 0x7f8756bbdb70> ====================
[2019-06-25 09:05:17,746] {fn_api_runner_transforms.py:490} INFO - ==================== <function impulse_to_input at 0x7f8756bbdbf8> ====================
[2019-06-25 09:05:17,748] {fn_api_runner_transforms.py:490} INFO - ==================== <function inject_timer_pcollections at 0x7f8756bbdd90> ====================
[2019-06-25 09:05:17,754] {fn_api_runner_transforms.py:490} INFO - ==================== <function sort_stages at 0x7f8756bbde18> ====================
[2019-06-25 09:05:17,758] {fn_api_runner_transforms.py:490} INFO - ==================== <function window_pcollection_coders at 0x7f8756bbdea0> ====================
[2019-06-25 09:05:17,787] {fn_api_runner.py:437} INFO - Running (((((((ref_AppliedPTransform_ReadData.eval/Read_106)+(ref_AppliedPTransform_DecodeData.eval/ParseTFExamples_108))+(ref_AppliedPTransform_GenerateStat

[2019-06-25 09:05:26,707] {fn_api_runner.py:437} INFO - Running (GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/Flatten/Read)+((GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/Precombine)+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/Group/Write))
[2019-06-25 09:05:26,891] {fn_api_runner.py:437} INFO - Running ((GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/Group/Read)+(GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/Merge))+((GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PostCombineFn)/ExtractOutputs)+((GenerateStatistics.train/RunStatsGenerators/GenerateSlicedStatisticsImpl/FlattenFe

[2019-06-25 09:05:27,434] {fn_api_runner.py:437} INFO - Running ((ref_PCollection_PCollection_55/Read)+(ref_AppliedPTransform_WriteStatsOutput.train/Write/WriteImpl/PreFinalize_103))+(ref_PCollection_PCollection_63/Write)
[2019-06-25 09:05:27,455] {fn_api_runner.py:437} INFO - Running (ref_PCollection_PCollection_55/Read)+(ref_AppliedPTransform_WriteStatsOutput.train/Write/WriteImpl/FinalizeWrite_104)
[2019-06-25 09:05:27,468] {filebasedsink.py:290} INFO - Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
[2019-06-25 09:05:27,572] {filebasedsink.py:327} INFO - Renamed 1 shards in 0.10 seconds.
[2019-06-25 09:05:27,595] {fn_api_runner.py:437} INFO - Running ((GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PreCombineFn)/Group/Read)+(GenerateStatistics.eval/RunStatsGenerators/GenerateSlicedStatisticsImpl/BasicStatsGenerator/CombinePerKey(PreCombineFn)/Merge))+((((GenerateStatistics.eval/Ru

...

[2019-06-25 09:07:33,577] {base_executor.py:74} INFO - Inputs for Executor is: {"model_exports": [{"artifact": {"uri": "/root/taxi/data/simple/trainer/current/", "properties": {"split": {"stringValue": ""}, "type_name": {"stringValue": "ModelExportPath"}}}, "artifact_type": {"name": "ModelExportPath", "properties": {"name": "STRING", "state": "STRING", "span": "INT", "split": "STRING", "type_name": "STRING"}}}], "examples": [{"artifact": {"uri": "/root/taxi/data/simple/csv_example_gen/train/", "properties": {"split": {"stringValue": "train"}, "type_name": {"stringValue": "ExamplesPath"}}}, "artifact_type": {"name": "ExamplesPath", "properties": {"name": "STRING", "state": "STRING", "span": "INT", "split": "STRING", "type_name": "STRING"}}}, {"artifact": {"uri": "/root/taxi/data/simple/csv_example_gen/eval/", "properties": {"split": {"stringValue": "eval"}, "type_name": {"stringValue": "ExamplesPath"}}}, "artifact_type": {"name": "ExamplesPath", "properties": {"name": "STRING", "state":

/usr/local/lib/python3.5/dist-packages/tensorflow_model_analysis/slicer/slicer.py:407: BeamDeprecationWarning: RemoveDuplicates is deprecated since 2.12. Use Distinct instead.
  | 'IncrementCounter' >> beam.Map(increment_counter))


[2019-06-25 09:07:35,528] {fn_api_runner_transforms.py:490} INFO - ==================== <function annotate_downstream_side_inputs at 0x7f8756bbd6a8> ====================
[2019-06-25 09:07:35,530] {fn_api_runner_transforms.py:490} INFO - ==================== <function fix_side_input_pcoll_coders at 0x7f8756bbd7b8> ====================
[2019-06-25 09:07:35,533] {fn_api_runner_transforms.py:490} INFO - ==================== <function lift_combiners at 0x7f8756bbd840> ====================
[2019-06-25 09:07:35,537] {fn_api_runner_transforms.py:490} INFO - ==================== <function expand_sdf at 0x7f8756bbd8c8> ====================
[2019-06-25 09:07:35,541] {fn_api_runner_transforms.py:490} INFO - ==================== <function expand_gbk at 0x7f8756bbd950> ====================
[2019-06-25 09:07:35,552] {fn_api_runner_transforms.py:490} INFO - ==================== <function sink_flattens at 0x7f8756bbda60> ====================
[2019-06-25 09:07:35,554] {fn_api_runner_transforms.py:490} I

[2019-06-25 09:08:02,913] {fn_api_runner.py:437} INFO - Running ((ExtractEvaluateAndWriteResults/WriteResults/WriteTFRecord(/root/taxi/data/simple/eval_output/plots)/WriteToTFRecord/Write/WriteImpl/GroupByKey/Read)+(ref_AppliedPTransform_ExtractEvaluateAndWriteResults/WriteResults/WriteTFRecord(/root/taxi/data/simple/eval_output/plots)/WriteToTFRecord/Write/WriteImpl/WriteBundles_92))+(ref_PCollection_PCollection_50/Write)
[2019-06-25 09:08:03,150] {fn_api_runner.py:437} INFO - Running ((ref_AppliedPTransform_ExtractEvaluateAndWriteResults/WriteEvalConfig(EvalConfig(model_location='/root/taxi/data/simple/trainer/current/eval_model_dir/1561453650', data_location='<user provided PCollection>', slice_spec=[SingleSliceSpec(columns=frozenset({'trip_start_hour'}), features=frozenset()), SingleSliceSpec(columns=frozenset(), features=frozenset())], example_weight_metric_key='post_export_metrics/example_count', num_bootstrap_samples=1))/CreateEvalConfig/Read_97)+(ref_AppliedPTransform_ExtractEv

[2019-06-25 09:08:03,370] {filebasedsink.py:290} INFO - Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
[2019-06-25 09:08:03,475] {filebasedsink.py:327} INFO - Renamed 1 shards in 0.10 seconds.
[2019-06-25 09:08:03,498] {fn_api_runner.py:437} INFO - Running (ExtractEvaluateAndWriteResults/WriteResults/WriteTFRecord(/root/taxi/data/simple/eval_output/metrics)/WriteToTFRecord/Write/WriteImpl/GroupByKey/Read)+((ref_AppliedPTransform_ExtractEvaluateAndWriteResults/WriteResults/WriteTFRecord(/root/taxi/data/simple/eval_output/metrics)/WriteToTFRecord/Write/WriteImpl/WriteBundles_76)+(ref_PCollection_PCollection_40/Write))
[2019-06-25 09:08:03,524] {fn_api_runner.py:437} INFO - Running ((ref_PCollection_PCollection_33/Read)+(ref_AppliedPTransform_ExtractEvaluateAndWriteResults/WriteResults/WriteTFRecord(/root/taxi/data/simple/eval_output/metrics)/WriteToTFRecord/Write/WriteImpl/PreFinalize_77))+(ref_PCollection_PCollection_41/Write)
[2019-06-25 09:

## Check Result

In [19]:
!ls -Rlhs /root/taxi/data/simple/

/root/taxi/data/simple/:
total 1.9M
4.0K drwxr-xr-x 4 root root 4.0K Jun 25 09:05 csv_example_gen
1.9M -rw-r--r-- 1 root root 1.9M Jun 25 09:05 data.csv
4.0K drwxr-xr-x 2 root root 4.0K Jun 25 09:08 eval_output
4.0K drwxr-xr-x 2 root root 4.0K Jun 25 09:05 schema_gen
4.0K drwxr-xr-x 4 root root 4.0K Jun 25 09:05 statistics_gen
4.0K drwxr-xr-x 3 root root 4.0K Jun 25 09:05 trainer
4.0K drwxr-xr-x 4 root root 4.0K Jun 25 09:05 transform

/root/taxi/data/simple/csv_example_gen:
total 8.0K
4.0K drwxr-xr-x 2 root root 4.0K Jun 25 09:05 eval
4.0K drwxr-xr-x 2 root root 4.0K Jun 25 09:05 train

/root/taxi/data/simple/csv_example_gen/eval:
total 204K
204K -rw-r--r-- 1 root root 201K Jun 25 09:05 data_tfrecord-00000-of-00001.gz

/root/taxi/data/simple/csv_example_gen/train:
total 408K
408K -rw-r--r-- 1 root root 405K Jun 25 09:05 data_tfrecord-00000-of-00001.gz

/root/taxi/data/simple/eval_output:
total 34M
4.0K -rw-r--r-- 1 root root  506 Jun 25 09:08 eval_config
 12K -rw-r--r-- 1 root root 8.

In [20]:
def get_eval_dir(model_analyzer):
    artifact = model_analyzer.outputs.output.get()
    return types.get_single_uri(artifact)
    
eval_dir = get_eval_dir(model_analyzer)

In [21]:
import tensorflow_model_analysis as tfma
result = tfma.load_eval_result(eval_dir)

In [22]:
tfma.view.render_slicing_metrics(result, slicing_column='trip_start_hour')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_count'}, data=[{'metrics':…

In [23]:
%config InlineBackend.figure_format = 'retina'
tfma.view.render_plot(result, tfma.slicer.SingleSliceSpec(features=[('trip_start_hour', 10)]))

PlotViewer(config={'metricKeys': {'calibrationPlot': {'dataSeries': 'buckets', 'metricName': 'calibrationHisto…